# Day 23

https://adventofcode.com/2020/day/23

## Part 1

I'll begin with a try-and-error solution based on a `list` of `int` and slicing and rearranging, but I'm sure I'll regret it later...

In [139]:
def roundCups(ic,_cups,verbose=True):
    
    cups = list(_cups)
    ic = ic%len(cups)
    cmin = min(cups)
    cmax = max(cups)
    if verbose: print("Cups:        ",cups)   
    c = cups[ic]
    if verbose: print("Current:      ",c)
    
    # rearrage cups so that current is first
    cups = cups[ic:]+cups[:ic]
    
    # pick cups
    picks = cups[1:4]
    if verbose: print("Picks:       ",picks)

    # find destination
    target = c-1
    if target < cmin:
        target = cmax
    while True:
        jc = cups.index(target)
        if cups[jc] in picks:
            target -= 1
            if target < cmin:
                target = cmax
        else:
            break
    if verbose: print("Destination:  ",target)

    # remove picks
    cups = [cups[0]]+cups[4:]
    
    # index of target after removing picks
    jt = cups.index(target)
    
    # insert picks back
    cups = cups[:jt+1]+picks+cups[jt+1:]
    
    # rearrage cups as a circular ring
    newcups = []
    j = len(cups)-ic
    while len(newcups)<len(cups):
        j = j%len(cups)
        newcups.append(cups[j])
        j += 1
    return newcups, (ic+1)%len(newcups)

In [140]:
cuplist = "389125467"
cups = [int(c) for c in cuplist]

for ic in range(10):
    print('==',ic+1,'==')
    newcups, current = roundCups(ic,cups)
    cups = newcups

print("== final ==")
print("Cups:        ",cups)
print("Current cup: ",cups[current])

== 1 ==
Cups:         [3, 8, 9, 1, 2, 5, 4, 6, 7]
Current:       3
Picks:        [8, 9, 1]
Destination:   2
== 2 ==
Cups:         [3, 2, 8, 9, 1, 5, 4, 6, 7]
Current:       2
Picks:        [8, 9, 1]
Destination:   7
== 3 ==
Cups:         [3, 2, 5, 4, 6, 7, 8, 9, 1]
Current:       5
Picks:        [4, 6, 7]
Destination:   3
== 4 ==
Cups:         [7, 2, 5, 8, 9, 1, 3, 4, 6]
Current:       8
Picks:        [9, 1, 3]
Destination:   7
== 5 ==
Cups:         [3, 2, 5, 8, 4, 6, 7, 9, 1]
Current:       4
Picks:        [6, 7, 9]
Destination:   3
== 6 ==
Cups:         [9, 2, 5, 8, 4, 1, 3, 6, 7]
Current:       1
Picks:        [3, 6, 7]
Destination:   9
== 7 ==
Cups:         [7, 2, 5, 8, 4, 1, 9, 3, 6]
Current:       9
Picks:        [3, 6, 7]
Destination:   8
== 8 ==
Cups:         [8, 3, 6, 7, 4, 1, 9, 2, 5]
Current:       2
Picks:        [5, 8, 3]
Destination:   1
== 9 ==
Cups:         [7, 4, 1, 5, 8, 3, 9, 2, 6]
Current:       6
Picks:        [7, 4, 1]
Destination:   5
== 10 ==
Cups:         [5, 7

In [142]:
#cuplist = "389125467" # example
cuplist = "326519478" # input

cups = [int(c) for c in cuplist]

for ic in range(100):
    newcups, current = roundCups(ic,cups,verbose=False)
    cups = newcups

print("== final ==")
print("Cups:        ",cups)
print("Current cup: ",cups[current])

i1 = cups.index(1)
l = cups[i1+1:]+cups[:i1]
labels = "".join([ str(c) for c in l ])
print("Labels:",labels)

== final ==
Cups:         [4, 7, 9, 1, 2, 5, 3, 6, 8]
Current cup:  7
Labels: 25368479


## Part 2

My implementation for Part 1 is very inefficient (too much copying and moving around), it cannot work for Part 2 given the problem size. Let's see whether I can improve it...

In [171]:
def moveCups(cups,rounds):
    
    cmin = min(cups)
    cmax = max(cups)
    
    for ir in range(rounds):

        if ir%100==0:
            print(ir)
        
        # always assume current cup is first in list, rearrage only at end of rounds
        ic = ir%len(cups)
        picks = cups[1:4]
        target = cups[0]-1
        if target < cmin:
            target = cmax
        while True:
            jc = cups.index(target)
            if cups[jc] in picks:
                target -= 1
                if target < cmin:
                    target = cmax
            else:
                break
        cups = [cups[0]]+cups[4:] # remove picks
        jt = cups.index(target)   # index of target after removing picks
        cups = cups[:jt+1]+picks+cups[jt+1:] # insert picks back    
        # move first cup to the end so that next current is first for next round
        cups = cups[1:]+[cups[0]]

    shift = rounds%len(cups)
    cups = cups[-shift:]+cups[:-shift]
    return cups

#cuplist = "389125467" # example
cuplist = "326519478" # input

cups = [int(c) for c in cuplist]

cups = moveCups(cups,100)

i1 = cups.index(1)
l = cups[i1+1:]+cups[:i1]
labels = "".join([ str(c) for c in l ])
print("Labels:",labels)

0
Labels: 25368479


In [193]:
cuplist = "389125467" # example
#cuplist = "326519478" # input

cups = [int(c) for c in cuplist]
morecups = cups + list(range(len(cups) + 1, 1000000 + 1))

#morecups = moveCups(morecups,10000000)

The above implementation is still too inefficient to deal with 10M iterations on a 1M-long cup list...

A `list` is probably not the right data structure for this problem, and slicing and recomposing is certainly not efficient enough. 

Moreover, maybe I don't need to move around the whole list? On the larger case of Part 2, only a tiny fraction of the numbers are moved around!

If I save the status of the number ring as a **dictionary of the number following a given number**, at each round **I will only need to update 3 numbers**!

Example, first round:

`N:   3 8 9 1 2 5 4 6 7` 

`N+1: 8 9 1 2 5 4 6 7 3` (`N+1` here means the number following `N`)

The current number is 3, the picks are (8,9,1), the destination is 2. After all shifting I get:

`N:   3 2 8 9 1 5 4 6 7` 

`N+1: 2 9 1 5 8 4 6 7 3`

In the "following numbers" line (`N+1`) only 3 values have changed:

* The `N+1` of the CURRENT number (3) has become the previous `N+1` of the LAST OF THE PICKS
* The `N+1` of the DESTINATION number (2) has become the `N+1` of CURRENT (e.g. FIRST OF PICKS)
* the `N+1` of the LAST OF THE PICKS (1) has become the previous `N+1` of the DESTINATION

Every other entry of the dictionary would not have changed! This should indeed be more efficient!

In [199]:
cnext = {c1: c2 for c1, c2 in zip(cups, cups[1:]+[cups[0]])}
cnext

{3: 8, 8: 9, 9: 1, 1: 2, 2: 5, 5: 4, 4: 6, 6: 7, 7: 3}

In [260]:
def moveCupsDict(cups, rounds):

    # dictionary of cups following each cup
    cnext = {c1: c2 for c1, c2 in zip(cups, cups[1:]+[cups[0]])}
    
    # current cup, begin with the initial in list
    curr = cups[0]
    cmin = min(cups)
    cmax = max(cups)
    
    for i in range(rounds):
        
        if i%1000000==0: print(i,"rounds...")
        
        # get picks from cnext dictionary
        c = curr
        picks = []
        for i in range(3):
            picks.append(cnext[c])
            c = cnext[c]
        #print(picks)
        
        # The search for the destination in my previous implementation was also VERY inefficient,
        # but I was using the list index() method without really needing to do so!
        # New one below is wat faster!
        target = curr-1
        if target < cmin:
            target = cmax
        while True:
            if target in picks:
                target -= 1
                if target < cmin:
                    target = cmax
            else:
                break             
        #print(target)
        
        # now I need to update the cnext dictionary according the above rules
        # all three values needs to be updated at once not to mix states!
        cnext[curr], cnext[target], cnext[picks[-1]] = cnext[picks[-1]], cnext[curr], cnext[target]
        #cnext[curr], cnext[target], cnext[picks[-1]] = cnext[picks[-1]], picks[0], cnext[target]
        
        # define new current cups, as next in ring of last current cup
        curr = cnext[curr]

    print(rounds,"rounds...")
    # after all rounds, return ring status starting from cups labeled 1
    c = 1
    return [c := cnext[c] for _ in cups]

In [261]:
# Test new implementation on Part 1

#cuplist = "389125467" # example
cuplist = "326519478" # input

cups = [int(c) for c in cuplist]

cups_part1 = moveCupsDict(cups, 100)
cups_part1

0 rounds...
100 rounds...


[2, 5, 3, 6, 8, 4, 7, 9, 1]

In [263]:
# Part 2 solution

#cuplist = "389125467" # example
cuplist = "326519478" # input

cups = [int(c) for c in cuplist]

# enlarge list of cups...
morecups = cups + list(range(len(cups) + 1, 1000000 + 1))

from timeit import default_timer as timer
start = timer()

cups_part2 = moveCupsDict(morecups, 10000000)

print("Solution Part 2:",cups_part2[0] * cups_part2[1])

stop = timer() 
print("Time elapsed =", int(stop - start), "s")

0 rounds...
1000000 rounds...
2000000 rounds...
3000000 rounds...
4000000 rounds...
5000000 rounds...
6000000 rounds...
7000000 rounds...
8000000 rounds...
9000000 rounds...
10000000 rounds...
Solution Part 2: 44541319250
Time elapsed = 13 s
